In [1]:
import numpy as np
import tqdm

### Create my synthetic dataset

In [2]:
np.set_printoptions(suppress=True, precision=4)
np.set_printoptions(linewidth=100, threshold=np.inf)
np.set_printoptions(formatter={'int': '{:5d}'.format})

In [3]:
num_states = 10
num_observations = 100

In [19]:
transition_probs = np.random.dirichlet(np.ones(num_states), size=num_states)
emission_probs = np.random.dirichlet(np.ones(num_observations), size=num_states)
initial_state_dist = np.ones(num_states) / num_states

In [20]:
transition_probs, emission_probs

(array([[0.0828, 0.0835, 0.2565, 0.0532, 0.1432, 0.1047, 0.0355, 0.0749, 0.0597, 0.1059],
        [0.2622, 0.0788, 0.0438, 0.0515, 0.2498, 0.128 , 0.0694, 0.0793, 0.0056, 0.0315],
        [0.3887, 0.0121, 0.066 , 0.0311, 0.1557, 0.0581, 0.1044, 0.0123, 0.08  , 0.0917],
        [0.002 , 0.0137, 0.0416, 0.2126, 0.0617, 0.0089, 0.3625, 0.1019, 0.0024, 0.1927],
        [0.1317, 0.1843, 0.0463, 0.0339, 0.023 , 0.2441, 0.0081, 0.0168, 0.1834, 0.1285],
        [0.068 , 0.0984, 0.0047, 0.1357, 0.1073, 0.0184, 0.1956, 0.1457, 0.172 , 0.0542],
        [0.2729, 0.0094, 0.0608, 0.2362, 0.0922, 0.0745, 0.1118, 0.0079, 0.0578, 0.0767],
        [0.0392, 0.1629, 0.1814, 0.018 , 0.1053, 0.0753, 0.0035, 0.1197, 0.2539, 0.0409],
        [0.0439, 0.0656, 0.0169, 0.3232, 0.0293, 0.1207, 0.0119, 0.0515, 0.0976, 0.2395],
        [0.0633, 0.1669, 0.0989, 0.0001, 0.119 , 0.0354, 0.0203, 0.1823, 0.2566, 0.0573]]),
 array([[0.0326, 0.0006, 0.011 , 0.0327, 0.0227, 0.002 , 0.0014, 0.0023, 0.0013, 0.017 , 0.0058,
 

In [6]:
# Simulate the HMM and generate strings (observation sequences)
def simulate_hmm(num_sequences, min_length, max_length, start_prob, trans_prob, emis_prob):
    sequences = []
    hidden_states = []
    for _ in range(num_sequences):
        sequence_length = np.random.randint(min_length, max_length+1)
        current_state = np.random.choice(num_states, p=start_prob)
        observation_sequence = []
        state_sequence = []
        for _ in range(sequence_length):
            state_sequence.append(current_state)
            observation = np.random.choice(num_observations, p=emis_prob[current_state])
            observation_sequence.append(observation)
            current_state = np.random.choice(num_states, p=trans_prob[current_state])
        sequences.append(observation_sequence)
        hidden_states.append(state_sequence)
    
    return sequences, hidden_states

In [21]:
syn_sequences, syn_hidden_states = simulate_hmm(
    num_sequences=50000,
    min_length=10,
    max_length=30,
    start_prob=initial_state_dist,
    trans_prob=transition_probs,
    emis_prob=emission_probs
)

In [8]:
def add_noise_to_states(hidden_states, flip_prob=0.1):
    noisy_hidden_states = []
    for sequence in hidden_states:
        noisy_sequence = []
        for state in sequence:
            if np.random.rand() < flip_prob:
                # Flip the state to a different random state
                possible_states = list(range(num_states))
                possible_states.remove(state)  # Remove the current state from possibilities
                new_state = np.random.choice(possible_states)
                noisy_sequence.append(new_state)
            else:
                noisy_sequence.append(state)
        noisy_hidden_states.append(noisy_sequence)
    return noisy_hidden_states

In [38]:
noisy_hidden_states = add_noise_to_states(syn_hidden_states, flip_prob=0.3)

In [39]:
print(transition_probs)
for seq in syn_sequences[:5]:
    print('[' + ', '.join(map(str, seq)) + ']')
for index in range(len(noisy_hidden_states[:5])):
    print('[' + ', '.join(map(str, syn_hidden_states[:5][index])) + ']')
    print('[' + ', '.join(map(str, noisy_hidden_states[:5][index])) + ']')
    print("--------------------")

[[0.0828 0.0835 0.2565 0.0532 0.1432 0.1047 0.0355 0.0749 0.0597 0.1059]
 [0.2622 0.0788 0.0438 0.0515 0.2498 0.128  0.0694 0.0793 0.0056 0.0315]
 [0.3887 0.0121 0.066  0.0311 0.1557 0.0581 0.1044 0.0123 0.08   0.0917]
 [0.002  0.0137 0.0416 0.2126 0.0617 0.0089 0.3625 0.1019 0.0024 0.1927]
 [0.1317 0.1843 0.0463 0.0339 0.023  0.2441 0.0081 0.0168 0.1834 0.1285]
 [0.068  0.0984 0.0047 0.1357 0.1073 0.0184 0.1956 0.1457 0.172  0.0542]
 [0.2729 0.0094 0.0608 0.2362 0.0922 0.0745 0.1118 0.0079 0.0578 0.0767]
 [0.0392 0.1629 0.1814 0.018  0.1053 0.0753 0.0035 0.1197 0.2539 0.0409]
 [0.0439 0.0656 0.0169 0.3232 0.0293 0.1207 0.0119 0.0515 0.0976 0.2395]
 [0.0633 0.1669 0.0989 0.0001 0.119  0.0354 0.0203 0.1823 0.2566 0.0573]]
[99, 44, 31, 31, 19, 81, 21, 60, 12, 31, 60, 53, 22, 93, 10, 64]
[0, 83, 13, 38, 86, 59, 37, 21, 25, 29, 85, 58, 83, 95, 33, 12, 5, 99, 81, 1, 9, 13, 32, 73, 60, 51, 29]
[43, 97, 87, 18, 11, 34, 37, 82, 51, 28, 50]
[23, 43, 33, 82, 76, 31, 51, 75, 87, 62, 86, 24, 52, 1

In [40]:
file_path = "../data/hmm_synthetic_dataset.npz"
seq_object = np.array(syn_sequences, dtype=object)
hid_object = np.array(syn_hidden_states, dtype=object)
noisy_hid_object = np.array(noisy_hidden_states, dtype=object)
trans_object = transition_probs
np.savez(file_path, observation=seq_object, real_hidden=hid_object, noisy_hidden=noisy_hid_object, real_trans=trans_object)

In [36]:
loaded_npz = np.load("../data/hmm_synthetic_dataset.npz", allow_pickle=True)
observations = list(loaded_npz['observation'])
hidden_states = list(loaded_npz['real_hidden'])
noi_hidden_states = list(loaded_npz['noisy_hidden'])
transition_dist = np.vstack(loaded_npz['real_trans'])

In [37]:
print(transition_dist)
for seq in observations[:5]:
    print('[' + ', '.join(map(str, seq)) + ']')
for index in range(len(hidden_states[:5])):
    print('[' + ', '.join(map(str, hidden_states[:5][index])) + ']')
    print('[' + ', '.join(map(str, noi_hidden_states[:5][index])) + ']')
    print("--------------------")

[[0.0828 0.0835 0.2565 0.0532 0.1432 0.1047 0.0355 0.0749 0.0597 0.1059]
 [0.2622 0.0788 0.0438 0.0515 0.2498 0.128  0.0694 0.0793 0.0056 0.0315]
 [0.3887 0.0121 0.066  0.0311 0.1557 0.0581 0.1044 0.0123 0.08   0.0917]
 [0.002  0.0137 0.0416 0.2126 0.0617 0.0089 0.3625 0.1019 0.0024 0.1927]
 [0.1317 0.1843 0.0463 0.0339 0.023  0.2441 0.0081 0.0168 0.1834 0.1285]
 [0.068  0.0984 0.0047 0.1357 0.1073 0.0184 0.1956 0.1457 0.172  0.0542]
 [0.2729 0.0094 0.0608 0.2362 0.0922 0.0745 0.1118 0.0079 0.0578 0.0767]
 [0.0392 0.1629 0.1814 0.018  0.1053 0.0753 0.0035 0.1197 0.2539 0.0409]
 [0.0439 0.0656 0.0169 0.3232 0.0293 0.1207 0.0119 0.0515 0.0976 0.2395]
 [0.0633 0.1669 0.0989 0.0001 0.119  0.0354 0.0203 0.1823 0.2566 0.0573]]
[99, 44, 31, 31, 19, 81, 21, 60, 12, 31, 60, 53, 22, 93, 10, 64]
[0, 83, 13, 38, 86, 59, 37, 21, 25, 29, 85, 58, 83, 95, 33, 12, 5, 99, 81, 1, 9, 13, 32, 73, 60, 51, 29]
[43, 97, 87, 18, 11, 34, 37, 82, 51, 28, 50]
[23, 43, 33, 82, 76, 31, 51, 75, 87, 62, 86, 24, 52, 1

### Initailise my model with pre-defined params

In [45]:
split = 5000
param_observed = observations[:split]
param_hidden = hidden_states[:split]
train_observed = observations[split:]
train_hidden = hidden_states[split:]

In [41]:
trans_count = np.zeros((num_states, num_states), dtype='int')
emis_count = np.zeros((num_observations, num_states), dtype='int')

In [88]:
for i in range(split):
    for t in range(len(param_hidden[i])):
        emis_count[param_observed[i][t], param_hidden[i][t]] += 1
        if t > 0: 
            trans_count[param_hidden[i][t], param_hidden[i][t - 1]] += 1

In [1]:
trans_count, emis_count

NameError: name 'trans_count' is not defined

### Test my model on the synthetic dataset